In [2]:
pip install torch torchvision torchaudio

Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'c:\Users\apoor\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [3]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.Resize((150, 150)),
    transforms.ToTensor()
])

train_data = datasets.ImageFolder("dataset/train", transform=transform)
val_data = datasets.ImageFolder("dataset/val", transform=transform)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)


In [4]:
import torch.nn as nn

class PneumoniaCNN(nn.Module):
    def __init__(self):
        super(PneumoniaCNN, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 32, 3), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(64, 128, 3), nn.ReLU(), nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(128 * 17 * 17, 128), nn.ReLU(),
            nn.Linear(128, 1), nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.model(x)


#Model Training

In [5]:
import torch
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = PneumoniaCNN().to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(10):
    model.train()
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.float().unsqueeze(1).to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}: Loss={loss.item():.4f}")


Epoch 1: Loss=0.2274
Epoch 2: Loss=0.1584
Epoch 3: Loss=0.0595
Epoch 4: Loss=0.1419
Epoch 5: Loss=0.0295
Epoch 6: Loss=0.0251
Epoch 7: Loss=0.0036
Epoch 8: Loss=0.0207
Epoch 9: Loss=0.0131
Epoch 10: Loss=0.0024


#Model validation

In [6]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for imgs, labels in val_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = model(imgs)
        preds = (outputs > 0.5).squeeze().long()
        correct += (preds == labels).sum().item()
        total += labels.size(0)
    print(f"Validation Accuracy: {(correct/total)*100:.2f}%")


Validation Accuracy: 100.00%


#save model

In [7]:
torch.save(model.state_dict(), "pneumonia_cnn_weights.pth")

In [2]:
import tkinter as tk
from tkinter import filedialog, Label
from PIL import Image, ImageTk
import torch
from torchvision import transforms
import os

# Define your model class again
import torch.nn as nn

class PneumoniaCNN(nn.Module):
    def __init__(self):
        super(PneumoniaCNN, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 32, 3), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(64, 128, 3), nn.ReLU(), nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(128 * 17 * 17, 128), nn.ReLU(),
            nn.Linear(128, 1), nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.model(x)

# Load model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = PneumoniaCNN()
model.load_state_dict(torch.load("pneumonia_cnn_weights.pth", map_location=device))
model.eval()

# Define transform
transform = transforms.Compose([
    transforms.Resize((150, 150)),
    transforms.ToTensor()
])

# GUI Function
def upload_image():
    file_path = filedialog.askopenfilename()
    if not file_path:
        return
    
    # Display image in GUI
    img = Image.open(file_path).convert("RGB")
    img_resized = img.resize((200, 200))
    img_tk = ImageTk.PhotoImage(img_resized)
    image_label.config(image=img_tk)
    image_label.image = img_tk
    
    # Preprocess and predict
    input_img = transform(img).unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(input_img)
        prediction = "PNEUMONIA" if output.item() > 0.5 else "NORMAL"
        result_label.config(text=f"Prediction: {prediction}", fg="green" if prediction == "NORMAL" else "red")

# Tkinter GUI
root = tk.Tk()
root.title("Pneumonia Detection")
root.geometry("400x400")

btn = tk.Button(root, text="Upload Chest X-Ray", command=upload_image)
btn.pack(pady=10)

image_label = Label(root)
image_label.pack()

result_label = Label(root, text="", font=("Arial", 16))
result_label.pack(pady=20)

root.mainloop()


In [ ]:
import tkinter as tk
from tkinter import filedialog, Label
from PIL import Image, ImageTk
import torch
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import os

class PneumoniaCNN(nn.Module):
    def __init__(self):
        super(PneumoniaCNN, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 32, 3), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(64, 128, 3), nn.ReLU(), nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(128 * 17 * 17, 128), nn.ReLU(),
            nn.Linear(128, 1), nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.model(x)

class PneumoniaCNNFeatures(nn.Module):
    def __init__(self, original_model):
        super(PneumoniaCNNFeatures, self).__init__()
        self.feature_extractor = nn.Sequential(*list(original_model.model.children())[:10])
    
    def forward(self, x):
        return self.feature_extractor(x)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = PneumoniaCNN()
try:
    model.load_state_dict(torch.load("pneumonia_cnn_weights.pth", map_location=device))
except FileNotFoundError:
    print("Error: 'pneumonia_cnn_weights.pth' not found. Ensure the file exists in the working directory.")
    exit(1)
model.eval()

feature_model = PneumoniaCNNFeatures(model)
feature_model.eval()

transform = transforms.Compose([
    transforms.Resize((150, 150)),
    transforms.ToTensor()
])

def compute_reference_features(data_loader, feature_model, device, num_samples=100):
    features = []
    feature_model.eval()
    with torch.no_grad():
        for i, (imgs, _) in enumerate(data_loader):
            if i * data_loader.batch_size >= num_samples:
                break
            imgs = imgs.to(device)
            feats = feature_model(imgs)  
            features.extend(feats.cpu().numpy())
    return np.array(features)

try:
    train_data = datasets.ImageFolder("dataset/train", transform=transform)
    train_loader = DataLoader(train_data, batch_size=32, shuffle=False)
except FileNotFoundError:
    print("Error: 'dataset/train' folder not found. Ensure the training data is available.")
    exit(1)

reference_features = compute_reference_features(train_loader, feature_model, device, num_samples=100)
mean_features = np.mean(reference_features, axis=0)  

def upload_image():
    file_path = filedialog.askopenfilename()
    if not file_path:
        return
    
    try:
        img = Image.open(file_path).convert("RGB")
    except Exception as e:
        result_label.config(text=f"Error loading image: {str(e)}", fg="red")
        return
    
    img_resized = img.resize((200, 200))
    img_tk = ImageTk.PhotoImage(img_resized)
    image_label.config(image=img_tk)
    image_label.image = img_tk
    
    input_img = transform(img).unsqueeze(0).to(device)
    with torch.no_grad():
        features = feature_model(input_img).cpu().numpy().flatten()  
        similarity = cosine_similarity([features], [mean_features])[0][0]
        
        # Debug: Print similarity score
        print(f"Similarity score: {similarity:.4f}")
        
        # Threshold for OOD detection
        similarity_threshold = 0.7  
        
        if similarity < similarity_threshold:
            result_label.config(text="Prediction: Unknown (not a chest X-ray)", fg="blue")
        else:
            output = model(input_img)
            prediction = "PNEUMONIA" if output.item() > 0.5 else "NORMAL"
            result_label.config(text=f"Prediction: {prediction} (Similarity: {similarity:.2f})", 
                               fg="green" if prediction == "NORMAL" else "red")

# Tkinter GUI
root = tk.Tk()
root.title("Pneumonia Detection")
root.geometry("400x400")

btn = tk.Button(root, text="Upload Chest X-Ray", command=upload_image)
btn.pack(pady=10)

image_label = Label(root)
image_label.pack()

result_label = Label(root, text="", font=("Arial", 16))
result_label.pack(pady=20)

root.mainloop()

Similarity score: 0.5022
Similarity score: 0.7509
Similarity score: 0.6645
Similarity score: 0.6060
Similarity score: 0.7148
Similarity score: 0.6978
Similarity score: 0.7509
Similarity score: 0.7756
Similarity score: 0.6645
Similarity score: 0.5022
Similarity score: 0.5558
Similarity score: 0.5197
Similarity score: 0.5106
Similarity score: 0.5514
